In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier # <-- Change 1: New Model Import
from sklearn.metrics import f1_score

In [2]:
# --- A. LOAD DATA AND SPLIT ---
try:
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths.")
    raise

# Define X/y, assuming 'patient_id' is NOT in df_train
X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (X_train is 80% of data, X_val is 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# --- B. PREPROCESSING PIPELINE ---
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, all_features)
    ],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# ----------------------------------------------------------------------
# C. TRAIN MODEL, PREDICT, AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 5. Train Neural Network (MLP) Model
nn_model = MLPClassifier( # <-- Change 2: Model Instantiation
    hidden_layer_sizes=(50, 20), 
    max_iter=500, 
    alpha=0.001, 
    random_state=42,
    early_stopping=True # Good practice for NNs
) 
nn_model.fit(X_train_processed, y_train)
print("✅ Neural Network (MLP) Model trained successfully.")

# Optional: Evaluate Baseline F1 Score on Validation Set
X_val_processed = preprocessor.transform(X_val)
y_pred_val = nn_model.predict(X_val_processed)
f1 = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (Baseline): {f1:.4f}")

# 6. Prepare Test Features
test_ids = pd.read_csv('data/ids.csv')
X_test_full = df_test

# Ensure the columns are in the same order as the training features
X_test_full = X_test_full[all_features]

# 7. Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# 8. Prediction
y_test_pred = nn_model.predict(X_test_processed)

# 9. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/nn_model.csv', index=False) 
print("\nFirst 5 predictions in submission:")
print(submission_df.head())

✅ Neural Network (MLP) Model trained successfully.
Validation F1 Score (Baseline): 0.6725

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              1
4       11155              1
